In [6]:
from utils import get_response

# **Intent Recognition for Service Routing using Chain-of-Thought Reasoning and Inflection AI API**

This notebook demonstrates how to implement **intent recognition** for a **Service Router** using **Chain-of-Thought (CoT) reasoning**. The goal is to classify user intents and generate structured responses that optimize service routing. 

## **Overview**

- Utilizes **Chain-of-Thought prompting** to enhance intent classification and decision-making.
- Employs a **guiding instruction prompt** for consistent and structured output.
- The model performs the following tasks:
  - Extracts **user intent** by identifying the appropriate **service category** based on query and intent history.
  - Provides a **reasoning trace** before making a classification.
  - Outputs results in **XML format** for downstream automation.
  - Utilizes the **`inflection_3_productivity` model**, optimized for task-specific classification like intent recognition.

By leveraging **structured prompting and reasoning**, this approach improves **accuracy** and ensures **efficient, interpretable routing**. The **XML output** format enables seamless integration with automation pipelines.

In [2]:
system_instruction_prompt = """
You are an AI assistant designed to determine the correct service intent for routing a user's query. Follow a step-by-step approach based on the sequence of prior intents and the user's current input.

## Your Task:
1. You will be provided with a sequence of previously identified service intents.
2. You will also be provided with the user's current query.
3. Your task is to analyze both pieces of information and determine the most appropriate next service intent.

# Service Intent Definitions:
- check_account_balance: Retrieve the user's account balance.
- transfer_money: Transfer funds between accounts.
- check_transaction_status: Check the status of a past transaction.
- search_nearby_restaurants: Find nearby restaurants.
- view_restaurant_reviews: Retrieve reviews for a restaurant.
- book_table: Make a reservation at a restaurant.
- search_flights: Look up available flights.
- select_flight: Choose a flight.
- enter_passenger_details: Provide passenger information for a flight.
- retrieve_flight_information: Get details about a flight.
- search_product: Find a product.
- add_to_cart: Add a product to the shopping cart.
- apply_discount: Apply a discount code to the cart.

# Output Format (XML):
You will respond in a valid XML format with the <parts>, <reasoning>, and <intent> tags following below output format. You don't need to provide explanation or any other information, just return reasoning and intent.
- Reasoning: A short explanation of why this intent was chosen.
- Intent: The determined service intent.

# Format of the Output
<parts>
    <reasoning>The reasoning behind selecting this intent</reasoning>
    <intent>determined_service</intent>
</parts>

"""

example_previous_intents_1 = "check_account_balance -> transfer_money"
example_previous_user_query_1 = "Can you confirm if the transfer was successful?"
example_output_1 = """
    <parts>
        <reasoning>Can you confirm if the transfer was successful?"
        example_reasoning_1 = "The user has already requested to transfer funds. Now, they are asking about the status of that transfer. This query is related to checking the transaction status.</reasoning>
        <intent>check_transaction_status</intent>
    </parts>
"""

example_previous_intents_2 = "search_nearby_restaurants -> view_restaurant_reviews"
example_previous_user_query_2 = "Can you book a table for 7 PM?"
example_output_2 = """
    <parts>
        <reasoning>The user started by searching for restaurants and then viewed reviews. Now, they are asking to reserve a table at one of the restaurants. This query is related to making a reservation.</reasoning>
        <intent>book_table</intent>
    </parts>
"""

example_previous_intents_3 = "search_flights -> select_flight -> enter_passenger_details"
example_previous_user_query_3 = "What are the baggage policies for this airline?"
example_output_3 = """
    <parts>
        <reasoning>What are the baggage policies for this airline?"
        example_reasoning_3 = "The user has already selected a flight and entered passenger details. Now, they are asking for additional information about the flight's baggage policy. This query is related to retrieving flight details.</reasoning>
        <intent>retrieve_flight_information</intent>
    </parts>
"""

example_previous_intents_4 = "search_product -> add_to_cart"
example_previous_user_query_4 = "Can you apply a discount code for me?"
example_output_4 = """
    <parts>
        <reasoning>The user has added a product to their cart and now wants to apply a discount. This query is related to applying a coupon or promotion.</reasoning>
        <intent>apply_discount</intent>
    </parts>
"""

In [3]:
def get_service_router_context(previous_intents: str, service_request: str) -> list:
    """
    Returns the context for the service router.
    """
    context = [
            {"type": "Instruction", "text": system_instruction_prompt},

            {"type": "Human", "text": example_previous_intents_1},
            {"type": "Human", "text": example_previous_user_query_1},
            {"type": "AI", "text": example_output_1},

            {"type": "Human", "text": example_previous_intents_2},
            {"type": "Human", "text": example_previous_user_query_2},
            {"type": "AI", "text": example_output_2},

            {"type": "Human", "text": example_previous_intents_3},
            {"type": "Human", "text": example_previous_user_query_3},
            {"type": "AI", "text": example_output_3},

            {"type": "Human", "text": example_previous_intents_4},
            {"type": "Human", "text": example_previous_user_query_4},
            {"type": "AI", "text": example_output_4},
            {
                "type": "Human",
                "text": f"Query: {previous_intents}",
            },
            {
                "type": "Human",
                "text": f"Query: {service_request}",
            },
        ]
    return context

## Test Scenario: Intent Recognition for Service Router

In [4]:
class color:
    BOLD = '\033[1m'
    END = '\033[0m'

In [5]:
async def test_service_router():
    print("Starting test: test_service_router")
    print("+*"*20)
    previous_intents = "search_nearby_restaurants"
    service_request = "Please show me the reviews."
    context = get_service_router_context(previous_intents, service_request)
    result = await get_response(context, ["reasoning", "intent"])

    assert result["intent"] == "view_restaurant_reviews"
    print(f"{color.BOLD} Intent recognized: {color.END} {result["intent"]}")
    print(f"{color.BOLD} Reasoning: {color.END} {result["reasoning"]}")
    print("+*"*20)

    previous_intents = "search_flights -> select_flight"
    service_request = "Used auto saved information to fill out the details."
    context = get_service_router_context(previous_intents, service_request)
    result = await get_response(context, ["reasoning", "intent"])

    assert result["intent"] == "enter_passenger_details"
    print(f"{color.BOLD} Intent recognized:{color.END} {result["intent"]}")
    print(f"{color.BOLD} Reasoning: {color.END} {result["reasoning"]}")
    print("+*"*20)

    previous_intents = "search_product -> add_to_cart"
    service_request = "I have a discount code ABCD1234. Please apply to this order."
    context = get_service_router_context(previous_intents, service_request)
    result = await get_response(context, ["reasoning", "intent"])

    assert result["intent"] == "apply_discount"
    print(f"{color.BOLD} Intent recognized: {color.END} {result["intent"]}")
    print(f"{color.BOLD} Reasoning: {color.END} {result["reasoning"]}")
    print("+*"*20)

    print("All tests passed successfully! 🙌")

# Run the test
await test_service_router()

Starting test: test_service_router
+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*


INFO:inference:Inflection AI API request took 10980.20 ms


 Intent recognized:  view_restaurant_reviews
 Reasoning:  The user started by searching for nearby restaurants. Now, they want to see reviews for the restaurants. This query is related to viewing restaurant reviews.
+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*


INFO:inference:Inflection AI API request took 10644.84 ms


 Intent recognized: enter_passenger_details
 Reasoning:  The user searched for flights and then selected one. They have auto-saved information to fill out the passenger details. This sequence implies that they have completed the passenger information step.
+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*


INFO:inference:Inflection AI API request took 4008.06 ms


 Intent recognized:  apply_discount
 Reasoning:  The user has provided a discount code and wants it to be applied to their order. This query is related to applying a coupon or promotion.
+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*
All tests passed successfully! 🙌
